# Electricity & Weather Data Analysis

# Introduction
This project is meant to gather insights on electricity usage.

This file looks for insights into electricity usage relative to weather data.

## Data
* This uses data cleaned with "green_button_data_cleaning.ipynb": clean_energy_use_*.csv
* This uses data cleaned with "weather_data_cleaning.ipynb.ipynb": clean_weather_*.csv


## Original Energy Data Source
Data is from my energy company(ComEd) from the past year. 10_22_2022 to 10_22_2023
Data from the [My Green Button](https://secure.comed.com/MyAccount/MyBillUsage/pages/secure/GreenButtonConnectDownloadMyData.aspx) webpage on the ComEd website.

## Original Weather Data Source
This data was collected using [Meteostat](https://github.com/meteostat/meteostat-python). The Meteostat Python library provides a simple API for accessing open weather and climate data. The historical observations and statistics are collected by Meteostat from different public interfaces, most of which are governmental.

Among the data sources are national weather services like the National Oceanic and Atmospheric Administration (NOAA) and Germany's national meteorological service (DWD).

# Column / header info


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Import the clean energy use spreadsheet from the 'data' directory

# Define the directory path and the regular expression pattern
import glob
directory_path = "./data"
file_pattern = "clean_energy_use*.csv"

# Use glob.glob to match filenames based on the pattern
file_path = glob.glob(f"{directory_path}/{file_pattern}")[0]
energy_df_raw = pd.read_csv(filepath_or_buffer=file_path)
energy_df_raw.info()
energy_df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17517 entries, 0 to 17516
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   START_TIME  17517 non-null  object 
 1   DATE        17517 non-null  object 
 2   END_TIME    17517 non-null  object 
 3   USAGE       17517 non-null  float64
 4   COST        17517 non-null  float64
 5   USAGE_DUR   17517 non-null  object 
dtypes: float64(2), object(4)
memory usage: 821.2+ KB


,START_TIME,DATE,END_TIME,USAGE,COST,USAGE_DUR
0,2022-10-22 00:00:00,2022-10-22 00:00:00,2022-10-22 00:29:00,0.11,0.01,0 days 00:29:00
1,2022-10-22 00:30:00,2022-10-22 00:00:00,2022-10-22 00:59:00,0.13,0.02,0 days 00:29:00
2,2022-10-22 01:00:00,2022-10-22 00:00:00,2022-10-22 01:29:00,0.09,0.01,0 days 00:29:00
3,2022-10-22 01:30:00,2022-10-22 00:00:00,2022-10-22 01:59:00,0.20,0.02,0 days 00:29:00
4,2022-10-22 02:00:00,2022-10-22 00:00:00,2022-10-22 02:29:00,0.10,0.01,0 days 00:29:00


In [3]:
# Import the energy use spreadsheet from the 'data' directory

# Define the directory path and the regular expression pattern
import glob
directory_path = "./data"
file_pattern = "clean_weather*.csv"

# Use glob.glob to match filenames based on the pattern
file_path = glob.glob(f"{directory_path}/{file_pattern}")[0]
weather_df_raw = pd.read_csv(filepath_or_buffer=file_path)
weather_df_raw.info()
weather_df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8785 entries, 0 to 8784
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    8785 non-null   object 
 1   temp    8785 non-null   float64
 2   dwpt    8785 non-null   float64
 3   rhum    8785 non-null   float64
 4   prcp    8785 non-null   float64
 5   wdir    8785 non-null   float64
 6   wspd    8785 non-null   float64
 7   pres    8785 non-null   float64
dtypes: float64(7), object(1)
memory usage: 549.2+ KB


,time,temp,dwpt,rhum,prcp,wdir,wspd,pres
0,2022-10-21 00:00:00,13.0,1.1,44.0,0.0,190.0,7.6,1008.0
1,2022-10-21 01:00:00,10.7,1.0,51.0,0.0,160.0,7.6,1008.0
2,2022-10-21 02:00:00,9.0,1.5,59.0,0.0,180.0,5.4,1008.0
3,2022-10-21 03:00:00,9.0,1.5,59.0,0.0,180.0,5.4,1008.0
4,2022-10-21 04:00:00,7.6,1.5,65.0,0.0,170.0,5.4,1008.0


## Observations & TODOs

- [ ] convert the time col (in both df's) into Datetime objs
- [ ] combine the half-hour data granularity of the energy_df into a df with 1hr rows
- [ ] Merge the dataset on the time start of each row
- [ ] correlate the energy usage with the temperature
- [ ] create an AI to predict energy usage based on weather data
- [ ] 
- [ ] 
- [ ] 

# Merged Column Descriptions

## energy_df
* **DATE**: Day recorded
* **START_TIME**: start of recording in Hour:Minutes
* **END_TIME**: end of recording in Hour:Minutes
* **USAGE**: Electric usage in kWh
* **COST**: amount charged for energy usage in USD

## weather_df
src: [Meteostat Documentation](https://dev.meteostat.net/python/hourly.html#data-structure)

| | | |
|-|-|-|
|**Column**|**Description**|**Type**|
|**time**|datetime of the observation|Datetime64|
|**temp**|air temperature in *°C*|Float64|
|**dwpt**|dew point in *°C*|Float64|
|**rhum**|relative humidity in percent (*%*)|Float64|
|**prcp**|one hour precipitation total in *mm*|Float64|
|**snow**|snow depth in *mm*|Float64|
|**wdir**|average wind direction in degrees (*°*)|Float64|
|**wspd**|average wind speed in *km/h*|Float64|
|**pres**|average sea-level air pressure in *hPa*|Float64|

# Hourly Usage Analysis

## Warm Months
It seems our electricity usage directly correlates with the sunlight hours of the day(8am-8pm). This makes sense for 2 reasons:
* We'd use the air conditioner for cooling during those hours.
* Those are the awake hours, so we'd use appliances during those hours.

## Cold Months
* Heating/Cooling USAGE should inversely correlate with sunlight hours.
* Other appliance USAGE still should correlate with sunlight hours.
These 2 USAGE timelines inversely correlate.


## Comparison
Cold months have about 3x the electricity USAGE than warm months. This likely has 2 causes.
* Chicago winters are much worse than the summers.
* Our heaters are MUCH less efficient than our air conditioners.